***GENERATED CODE FOR hybvselectricclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Vehicle ID', 'transformation_label': 'String Indexer'}], 'feature': 'Vehicle ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '13376.33', 'stddev': '150.46', 'min': ',AWD,11,230,3,29,497,697,111,77,5', 'max': '\\"', 'missing': '0', 'distinct': '481'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Vehicle ID'}, {'feature_label': 'Vehicle ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Vehicle ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Fuel ID', 'transformation_label': 'String Indexer'}], 'feature': 'Fuel ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '497', 'mean': '43.46', 'stddev': '8.13', 'min': '11', 'max': 'Volvo', 'missing': '3', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Fuel ID'}, {'feature_label': 'Fuel ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Fuel ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Fuel Configuration ID', 'transformation_label': 'String Indexer'}], 'feature': 'Fuel Configuration ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '487', 'mean': '5.59', 'stddev': '3.9', 'min': '1', 'max': 'https://www.mbusa.com', 'missing': '13', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Fuel Configuration ID'}, {'feature_label': 'Fuel Configuration ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Fuel Configuration ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Manufacturer ID', 'transformation_label': 'String Indexer'}], 'feature': 'Manufacturer ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '296.05', 'stddev': '85.78', 'min': '209', 'max': 'Van', 'missing': '1', 'distinct': '45'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Manufacturer ID'}, {'feature_label': 'Manufacturer ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Manufacturer ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Category ID', 'transformation_label': 'String Indexer'}], 'feature': 'Category ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '27.56', 'stddev': '2.48', 'min': '25', 'max': 'PHEV', 'missing': '1', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Category ID'}, {'feature_label': 'Category ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Category ID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Model', 'transformation_label': 'String Indexer'}], 'feature': 'Model', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '499', 'mean': '200.0', 'stddev': '0.0', 'min': '"RZ 450e AWD (18""/20"" Wheels)"', 'max': 'iX xDrive50', 'missing': '1', 'distinct': '341'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Model'}, {'feature_label': 'Model', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Model')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Model Year', 'transformation_label': 'String Indexer'}], 'feature': 'Model Year', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '498', 'mean': '2014.67', 'stddev': '133.67', 'min': '2023', 'max': 'Hybrid Electric', 'missing': '2', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Model Year'}, {'feature_label': 'Model Year', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Model Year')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Alternative Fuel Economy City', 'transformation_label': 'String Indexer'}], 'feature': 'Alternative Fuel Economy City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '128', 'mean': '71.38', 'stddev': '43.15', 'min': '100', 'max': '99', 'missing': '372', 'distinct': '65'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Alternative Fuel Economy ...'}, {'feature_label': 'Alternative Fuel Economy City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Alternative Fuel Economy City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Alternative Fuel Economy Highway', 'transformation_label': 'String Indexer'}], 'feature': 'Alternative Fuel Economy Highway', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '129', 'mean': '201.59', 'stddev': '179.12', 'min': '100', 'max': '99', 'missing': '371', 'distinct': '61'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Alternative Fuel Economy ...'}, {'feature_label': 'Alternative Fuel Economy Highway', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Alternative Fuel Economy Highway')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Alternative Fuel Economy Combined', 'transformation_label': 'String Indexer'}], 'feature': 'Alternative Fuel Economy Combined', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '248', 'mean': '82.65', 'stddev': '46.59', 'min': '100-109', 'max': 'Blended', 'missing': '252', 'distinct': '119'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Alternative Fuel Economy ...'}, {'feature_label': 'Alternative Fuel Economy Combined', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Alternative Fuel Economy Combined')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Conventional Fuel Economy City', 'transformation_label': 'String Indexer'}], 'feature': 'Conventional Fuel Economy City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '252', 'mean': '25.84', 'stddev': '24.49', 'min': '12', 'max': 'Includes standard and Sport models', 'missing': '248', 'distinct': '35'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Conventional Fuel Economy...'}, {'feature_label': 'Conventional Fuel Economy City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Conventional Fuel Economy City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Conventional Fuel Economy Highway', 'transformation_label': 'String Indexer'}], 'feature': 'Conventional Fuel Economy Highway', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '293', 'mean': '28.54', 'stddev': '9.58', 'min': '111', 'max': 'RWD', 'missing': '207', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Conventional Fuel Economy...'}, {'feature_label': 'Conventional Fuel Economy Highway', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Conventional Fuel Economy Highway')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Conventional Fuel Economy Combined', 'transformation_label': 'String Indexer'}], 'feature': 'Conventional Fuel Economy Combined', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '349', 'mean': '23.18', 'stddev': '8.85', 'min': '11', 'max': '9', 'missing': '151', 'distinct': '74'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Conventional Fuel Economy...'}, {'feature_label': 'Conventional Fuel Economy Combined', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Conventional Fuel Economy Combined')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Transmission Type', 'transformation_label': 'String Indexer'}], 'feature': 'Transmission Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '457', 'mean': '81.33', 'stddev': '102.77', 'min': '200', 'max': 'Automatic', 'missing': '43', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Transmission Type'}, {'feature_label': 'Transmission Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Transmission Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Engine Type', 'transformation_label': 'String Indexer'}], 'feature': 'Engine Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '484', 'mean': '2.97', 'stddev': '0.93', 'min': '2', 'max': 'e-motor', 'missing': '16', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Engine Type'}, {'feature_label': 'Engine Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Engine Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Engine Size', 'transformation_label': 'String Indexer'}], 'feature': 'Engine Size', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '486', 'mean': '13.94', 'stddev': '6.63', 'min': '1.3L', 'max': 'TBD', 'missing': '14', 'distinct': '119'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Engine Size'}, {'feature_label': 'Engine Size', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Engine Size')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Engine/Motor(s) Description', 'transformation_label': 'String Indexer'}], 'feature': 'Engine/Motor(s) Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '359.51', 'stddev': '60.52', 'min': '1.3L I4;', 'max': 'TBD', 'missing': '5', 'distinct': '132'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Engine/Motor(s) Descripti...'}, {'feature_label': 'Engine/Motor(s) Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Engine/Motor(s) Description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Manufacturer', 'transformation_label': 'String Indexer'}], 'feature': 'Manufacturer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '450', 'mean': '50.32', 'stddev': '16.21', 'min': '111', 'max': 'Volvo', 'missing': '50', 'distinct': '50'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Manufacturer'}, {'feature_label': 'Manufacturer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Manufacturer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Manufacturer URL', 'transformation_label': 'String Indexer'}], 'feature': 'Manufacturer URL', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '405', 'mean': '18.1', 'stddev': '10.65', 'min': '10', 'max': 'https://www.mbusa.com', 'missing': '95', 'distinct': '43'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Manufacturer URL'}, {'feature_label': 'Manufacturer URL', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Manufacturer URL')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Category', 'transformation_label': 'String Indexer'}], 'feature': 'Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '441', 'mean': '5.34', 'stddev': '1.0', 'min': '2', 'max': 'Van', 'missing': '59', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Category'}, {'feature_label': 'Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Fuel Code', 'transformation_label': 'String Indexer'}], 'feature': 'Fuel Code', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '409', 'mean': '', 'stddev': '', 'min': 'BD', 'max': 'PHEV', 'missing': '91', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Fuel Code'}, {'feature_label': 'Fuel Code', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Fuel Code')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Fuel', 'transformation_label': 'String Indexer'}], 'feature': 'Fuel', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '409', 'mean': '', 'stddev': '', 'min': 'Biodiesel (B20)', 'max': 'Plug-in Hybrid Electric', 'missing': '91', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Fuel'}, {'feature_label': 'Fuel', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Fuel')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Fuel Configuration Name', 'transformation_label': 'String Indexer'}], 'feature': 'Fuel Configuration Name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '406', 'mean': '', 'stddev': '', 'min': 'Dedicated', 'max': 'Hybrid Electric', 'missing': '94', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Fuel Configuration Name'}, {'feature_label': 'Fuel Configuration Name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Fuel Configuration Name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Electric-Only Range', 'transformation_label': 'String Indexer'}], 'feature': 'Electric-Only Range', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '147', 'mean': '229.17', 'stddev': '106.42', 'min': '100', 'max': '9', 'missing': '353', 'distinct': '112'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Electric-Only Range'}, {'feature_label': 'Electric-Only Range', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Electric-Only Range')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PHEV Type', 'transformation_label': 'String Indexer'}], 'feature': 'PHEV Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '13', 'mean': '', 'stddev': '', 'min': 'Blended', 'max': 'Blended', 'missing': '487', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PHEV Type'}, {'feature_label': 'PHEV Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PHEV Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Notes', 'transformation_label': 'String Indexer'}], 'feature': 'Notes', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '100', 'mean': '', 'stddev': '', 'min': '"21"" wheels"', 'max': 'Single motor', 'missing': '400', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Notes'}, {'feature_label': 'Notes', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Notes')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Drivetrain', 'transformation_label': 'String Indexer'}], 'feature': 'Drivetrain', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '408', 'mean': '', 'stddev': '', 'min': '  21""', 'max': 'RWD', 'missing': '92', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Drivetrain'}, {'feature_label': 'Drivetrain', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Drivetrain')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Charging Rate Level 2 (kW)', 'transformation_label': 'String Indexer'}], 'feature': 'Charging Rate Level 2 (kW)', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '114', 'mean': '10.09', 'stddev': '2.72', 'min': ' and 21"" wheels"', 'max': '9.6-19.2', 'missing': '386', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Charging Rate Level 2 (kW...'}, {'feature_label': 'Charging Rate Level 2 (kW)', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Charging Rate Level 2 (kW)')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Charging Rate DC Fast (kW)', 'transformation_label': 'String Indexer'}], 'feature': 'Charging Rate DC Fast (kW)', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '106', 'mean': '179.72', 'stddev': '62.99', 'min': '100', 'max': 'RWD', 'missing': '394', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Charging Rate DC Fast (kW...'}, {'feature_label': 'Charging Rate DC Fast (kW)', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Charging Rate DC Fast (kW)')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Charging Speed Level 2 (miles added per hour of charging)', 'transformation_label': 'String Indexer'}], 'feature': 'Charging Speed Level 2 (miles added per hour of charging)', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '110', 'mean': '42.32', 'stddev': '51.0', 'min': '11', 'max': '8', 'missing': '390', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Charging Speed Level 2 (m...'}, {'feature_label': 'Charging Speed Level 2 (miles added per hour of charging)', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Charging Speed Level 2 (miles added per hour of charging)')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Seating Capacity', 'transformation_label': 'String Indexer'}], 'feature': 'Seating Capacity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '111', 'mean': '30.86', 'stddev': '76.84', 'min': '150', 'max': '7', 'missing': '389', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Seating Capacity'}, {'feature_label': 'Seating Capacity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Seating Capacity')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run hybvselectricclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	lightdutyvehicles = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/light-duty-vehicles-2024-02-26.csv', 'filename': 'light-duty-vehicles-2024-02-26.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(lightdutyvehicles)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run hybvselectricclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	hybvselectricclassificationautofe = TransformationMain.run(lightdutyvehicles,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Vehicle ID", "transformation_label": "String Indexer"}], "feature": "Vehicle ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "13376.33", "stddev": "150.46", "min": ",AWD,11,230,3,29,497,697,111,77,5", "max": "\\\"", "missing": "0", "distinct": "481"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Vehicle ID"}, {"transformationsData": [{"feature_label": "Fuel ID", "transformation_label": "String Indexer"}], "feature": "Fuel ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "497", "mean": "43.46", "stddev": "8.13", "min": "11", "max": "Volvo", "missing": "3", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Fuel ID"}, {"transformationsData": [{"feature_label": "Fuel Configuration ID", "transformation_label": "String Indexer"}], "feature": "Fuel Configuration ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "487", "mean": "5.59", "stddev": "3.9", "min": "1", "max": "https://www.mbusa.com", "missing": "13", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Fuel Configuration ID"}, {"transformationsData": [{"feature_label": "Manufacturer ID", "transformation_label": "String Indexer"}], "feature": "Manufacturer ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "296.05", "stddev": "85.78", "min": "209", "max": "Van", "missing": "1", "distinct": "45"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Manufacturer ID"}, {"transformationsData": [{"feature_label": "Category ID", "transformation_label": "String Indexer"}], "feature": "Category ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "27.56", "stddev": "2.48", "min": "25", "max": "PHEV", "missing": "1", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Category ID"}, {"transformationsData": [{"feature_label": "Model", "transformation_label": "String Indexer"}], "feature": "Model", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "200.0", "stddev": "0.0", "min": "\"RZ 450e AWD (18\"\"/20\"\" Wheels)\"", "max": "iX xDrive50", "missing": "1", "distinct": "341"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Model"}, {"transformationsData": [{"feature_label": "Model Year", "transformation_label": "String Indexer"}], "feature": "Model Year", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "498", "mean": "2014.67", "stddev": "133.67", "min": "2023", "max": "Hybrid Electric", "missing": "2", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Model Year"}, {"transformationsData": [{"feature_label": "Alternative Fuel Economy City", "transformation_label": "String Indexer"}], "feature": "Alternative Fuel Economy City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "128", "mean": "71.38", "stddev": "43.15", "min": "100", "max": "99", "missing": "372", "distinct": "65"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Alternative Fuel Economy ..."}, {"transformationsData": [{"feature_label": "Alternative Fuel Economy Highway", "transformation_label": "String Indexer"}], "feature": "Alternative Fuel Economy Highway", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "129", "mean": "201.59", "stddev": "179.12", "min": "100", "max": "99", "missing": "371", "distinct": "61"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Alternative Fuel Economy ..."}, {"transformationsData": [{"feature_label": "Alternative Fuel Economy Combined", "transformation_label": "String Indexer"}], "feature": "Alternative Fuel Economy Combined", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "248", "mean": "82.65", "stddev": "46.59", "min": "100-109", "max": "Blended", "missing": "252", "distinct": "119"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Alternative Fuel Economy ..."}, {"transformationsData": [{"feature_label": "Conventional Fuel Economy City", "transformation_label": "String Indexer"}], "feature": "Conventional Fuel Economy City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "252", "mean": "25.84", "stddev": "24.49", "min": "12", "max": "Includes standard and Sport models", "missing": "248", "distinct": "35"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Conventional Fuel Economy..."}, {"transformationsData": [{"feature_label": "Conventional Fuel Economy Highway", "transformation_label": "String Indexer"}], "feature": "Conventional Fuel Economy Highway", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "293", "mean": "28.54", "stddev": "9.58", "min": "111", "max": "RWD", "missing": "207", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Conventional Fuel Economy..."}, {"transformationsData": [{"feature_label": "Conventional Fuel Economy Combined", "transformation_label": "String Indexer"}], "feature": "Conventional Fuel Economy Combined", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "349", "mean": "23.18", "stddev": "8.85", "min": "11", "max": "9", "missing": "151", "distinct": "74"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Conventional Fuel Economy..."}, {"transformationsData": [{"feature_label": "Transmission Type", "transformation_label": "String Indexer"}], "feature": "Transmission Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "457", "mean": "81.33", "stddev": "102.77", "min": "200", "max": "Automatic", "missing": "43", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Transmission Type"}, {"transformationsData": [{"feature_label": "Engine Type", "transformation_label": "String Indexer"}], "feature": "Engine Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "484", "mean": "2.97", "stddev": "0.93", "min": "2", "max": "e-motor", "missing": "16", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Engine Type"}, {"transformationsData": [{"feature_label": "Engine Size", "transformation_label": "String Indexer"}], "feature": "Engine Size", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "486", "mean": "13.94", "stddev": "6.63", "min": "1.3L", "max": "TBD", "missing": "14", "distinct": "119"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Engine Size"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Engine Cylinder Count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "318", "mean": "7.35", "stddev": "29.35", "min": "3", "max": "527", "missing": "182"}, "updatedLabel": "Engine Cylinder Count"}, {"transformationsData": [{"feature_label": "Engine/Motor(s) Description", "transformation_label": "String Indexer"}], "feature": "Engine/Motor(s) Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "359.51", "stddev": "60.52", "min": "1.3L I4;", "max": "TBD", "missing": "5", "distinct": "132"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Engine/Motor(s) Descripti..."}, {"transformationsData": [{"feature_label": "Manufacturer", "transformation_label": "String Indexer"}], "feature": "Manufacturer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "450", "mean": "50.32", "stddev": "16.21", "min": "111", "max": "Volvo", "missing": "50", "distinct": "50"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Manufacturer"}, {"transformationsData": [{"feature_label": "Manufacturer URL", "transformation_label": "String Indexer"}], "feature": "Manufacturer URL", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "405", "mean": "18.1", "stddev": "10.65", "min": "10", "max": "https://www.mbusa.com", "missing": "95", "distinct": "43"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Manufacturer URL"}, {"transformationsData": [{"feature_label": "Category", "transformation_label": "String Indexer"}], "feature": "Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "441", "mean": "5.34", "stddev": "1.0", "min": "2", "max": "Van", "missing": "59", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Category"}, {"transformationsData": [{"feature_label": "Fuel Code", "transformation_label": "String Indexer"}], "feature": "Fuel Code", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "409", "mean": "", "stddev": "", "min": "BD", "max": "PHEV", "missing": "91", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Fuel Code"}, {"transformationsData": [{"feature_label": "Fuel", "transformation_label": "String Indexer"}], "feature": "Fuel", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "409", "mean": "", "stddev": "", "min": "Biodiesel (B20)", "max": "Plug-in Hybrid Electric", "missing": "91", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Fuel"}, {"transformationsData": [{"feature_label": "Fuel Configuration Name", "transformation_label": "String Indexer"}], "feature": "Fuel Configuration Name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "406", "mean": "", "stddev": "", "min": "Dedicated", "max": "Hybrid Electric", "missing": "94", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Fuel Configuration Name"}, {"transformationsData": [{"feature_label": "Electric-Only Range", "transformation_label": "String Indexer"}], "feature": "Electric-Only Range", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "147", "mean": "229.17", "stddev": "106.42", "min": "100", "max": "9", "missing": "353", "distinct": "112"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Electric-Only Range"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total Range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "13", "mean": "329.23", "stddev": "38.4", "min": "300", "max": "420", "missing": "487"}, "updatedLabel": "Total Range"}, {"transformationsData": [{"feature_label": "PHEV Type", "transformation_label": "String Indexer"}], "feature": "PHEV Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "13", "mean": "", "stddev": "", "min": "Blended", "max": "Blended", "missing": "487", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PHEV Type"}, {"transformationsData": [{"feature_label": "Notes", "transformation_label": "String Indexer"}], "feature": "Notes", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "100", "mean": "", "stddev": "", "min": "\"21\"\" wheels\"", "max": "Single motor", "missing": "400", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Notes"}, {"transformationsData": [{"feature_label": "Drivetrain", "transformation_label": "String Indexer"}], "feature": "Drivetrain", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "408", "mean": "", "stddev": "", "min": "  21\"\"", "max": "RWD", "missing": "92", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Drivetrain"}, {"transformationsData": [{"feature_label": "Charging Rate Level 2 (kW)", "transformation_label": "String Indexer"}], "feature": "Charging Rate Level 2 (kW)", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "114", "mean": "10.09", "stddev": "2.72", "min": " and 21\"\" wheels\"", "max": "9.6-19.2", "missing": "386", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Charging Rate Level 2 (kW..."}, {"transformationsData": [{"feature_label": "Charging Rate DC Fast (kW)", "transformation_label": "String Indexer"}], "feature": "Charging Rate DC Fast (kW)", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "106", "mean": "179.72", "stddev": "62.99", "min": "100", "max": "RWD", "missing": "394", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Charging Rate DC Fast (kW..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Charging Speed Level 1 (miles added per hour of charging)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "110", "mean": "3.96", "stddev": "2.73", "min": "2.0", "max": "19.0", "missing": "390"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Charging Speed Level 1 (m..."}, {"transformationsData": [{"feature_label": "Charging Speed Level 2 (miles added per hour of charging)", "transformation_label": "String Indexer"}], "feature": "Charging Speed Level 2 (miles added per hour of charging)", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "110", "mean": "42.32", "stddev": "51.0", "min": "11", "max": "8", "missing": "390", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Charging Speed Level 2 (m..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Charging Speed DC Fast (miles added per hour of charging)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "104", "mean": "359.55", "stddev": "177.23", "min": "3", "max": "876", "missing": "396"}, "updatedLabel": "Charging Speed DC Fast (m..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Battery Voltage", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "142", "mean": "438.24", "stddev": "189.46", "min": "22", "max": "850", "missing": "358"}, "updatedLabel": "Battery Voltage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Battery Capacity Amp Hours", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "142", "mean": "215.72", "stddev": "137.36", "min": "21", "max": "876", "missing": "358"}, "updatedLabel": "Battery Capacity Amp Hour..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Battery Capacity kWh", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "146", "mean": "115.51", "stddev": "116.63", "min": "6", "max": "800", "missing": "354"}, "updatedLabel": "Battery Capacity kWh"}, {"transformationsData": [{"feature_label": "Seating Capacity", "transformation_label": "String Indexer"}], "feature": "Seating Capacity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "111", "mean": "30.86", "stddev": "76.84", "min": "150", "max": "7", "missing": "389", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Seating Capacity"}]}))

	#transformationPostExecutionHook(hybvselectricclassificationautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run hybvselectricclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(hybvselectricclassificationautofe, ["Engine Cylinder Count", "Total Range", "Charging Speed Level 1 (miles added per hour of charging)", "Charging Speed DC Fast (miles added per hour of charging)", "Battery Voltage", "Battery Capacity Amp Hours", "Vehicle ID_stringindexer", "Fuel ID_stringindexer", "Fuel Configuration ID_stringindexer", "Manufacturer ID_stringindexer", "Category ID_stringindexer", "Model_stringindexer", "Model Year_stringindexer", "Alternative Fuel Economy City_stringindexer", "Alternative Fuel Economy Highway_stringindexer", "Alternative Fuel Economy Combined_stringindexer", "Conventional Fuel Economy City_stringindexer", "Conventional Fuel Economy Highway_stringindexer", "Conventional Fuel Economy Combined_stringindexer", "Transmission Type_stringindexer", "Engine Type_stringindexer", "Engine Size_stringindexer", "Engine/Motor(s) Description_stringindexer", "Manufacturer_stringindexer", "Manufacturer URL_stringindexer", "Category_stringindexer", "Fuel Code_stringindexer", "Fuel_stringindexer", "Fuel Configuration Name_stringindexer", "Electric-Only Range_stringindexer", "PHEV Type_stringindexer", "Notes_stringindexer", "Drivetrain_stringindexer", "Charging Rate Level 2 (kW)_stringindexer", "Charging Rate DC Fast (kW)_stringindexer", "Charging Speed Level 2 (miles added per hour of charging)_stringindexer", "Seating Capacity_stringindexer"], "Battery Capacity kWh")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

